# Load Dataset

In [99]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls")
dataset=dataset[dataset["pCR (outcome)"]!=999]
dataset.head(len(dataset))

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.000000,41.0,0,0.0,0.0,1.0,3,3,...,0.517172,0.375126,3.325332,0.002314,3.880772e+06,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.000000,39.0,1,1.0,0.0,0.0,3,3,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.000000,31.0,0,0.0,0.0,1.0,2,1,...,0.534549,0.534549,2.485848,0.006752,1.540027e+06,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.000000,35.0,0,0.0,0.0,1.0,3,3,...,0.506185,0.506185,2.606255,0.003755,6.936741e+06,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.000000,61.0,1,0.0,0.0,0.0,2,1,...,0.462282,0.462282,2.809279,0.006521,1.265399e+06,39.621023,0.006585,0.034148,0.001083,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,TRG002955,1,49.250000,46.1,0,0.0,0.0,1.0,3,3,...,0.439568,0.439568,3.056046,0.001339,1.671271e+07,79.989003,0.003282,0.024716,0.000812,0.003078
396,TRG002958,0,48.500000,53.3,0,0.0,0.0,1.0,2,1,...,0.527779,0.527778,1.500000,0.003728,2.132007e+05,0.996746,0.252582,0.007380,0.000037,0.231059
397,TRG002961,0,47.500000,68.8,1,0.0,0.0,0.0,3,3,...,0.313693,0.313693,3.573557,0.001112,2.008034e+07,204.864200,0.001372,0.054063,0.003697,0.001368
398,TRG002962,0,46.916667,46.0,1,0.0,0.0,0.0,2,1,...,0.670229,0.670229,1.857045,0.006706,5.609262e+05,9.609163,0.026591,0.018682,0.000311,0.022676


In [100]:
# from sklearn.impute import  SimpleImputer
# imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# names=dataset.columns.drop('ID')
# a=imp.fit_transform(dataset.drop('ID',axis=1))
# imp=SimpleImputer(missing_values=999,strategy='mean')
# a=imp.fit_transform(a)
# df=pd.DataFrame(a,columns=
#                 names)
# df

# Data Preprocessing

### Removing duplicate

In [101]:
dataset.drop_duplicates(inplace=True) #because with 119 features, it is unlikely that exact same data happens in real world, it is useless for training and testing; hence remove it before dataset split

### Split Dataset

In [102]:
dataset['pCR (outcome)'].value_counts()

0    299
1     96
Name: pCR (outcome), dtype: int64

In [103]:
y=dataset['pCR (outcome)']
X=dataset.drop(['pCR (outcome)'],axis=1)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=1)

### drop columns that are not applicable

In [104]:
class columnDropperTransformer(): #https://stackoverflow.com/questions/68402691/adding-dropping-column-instance-into-a-pipeline
    def __init__(self,columns):
        self.columns=columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit_transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

dropper = columnDropperTransformer(['ID'])
X_train=dropper.fit_transform(X_train)

X_test=dropper.transform(X_test)

### missing data

In [105]:
dataset.isnull().sum()

ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 120, dtype: int64

In [106]:
from sklearn.impute import KNNImputer

names=X_train.columns

imp=KNNImputer(missing_values=np.nan,n_neighbors=10)
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

imp.missing_values=np.float64(999)
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

### Removing outliers

In [107]:
from sklearn.neighbors import LocalOutlierFactor

# because use LocalOutlierFactor may lose useful information becuase whenever there is an outlier in any features of the sample, the sample will be detected as outlier sample and all its information will be set to 999
# therefore, use other method

class outlierHandlingWithDev():
    def fit(self,X,y=None):
        self.mean=np.zeros(len(X[0]))
        self.std=np.zeros(len(X[0]))
        for i in range(len(X[0])):
            self.mean[i]=X[:,i].mean()
            self.std[i]=X[:,i].std()

        return self

    def transform(self,X,y=None):
        for i in range(len(X)):
            for j in range(len(X[i])):
                if np.abs(X[i][j]-self.mean[j])>3*self.std[j]:
                    X[i][j]=999
        return X

    def fit_transform(self,X,y=None):
        self.fit(X,y)
        return self.transform(X,y)
    # def fit(self,X,y=None):
    #     self.mean=X.mean(axis=0)
    #     self.std=X.std(axis=0)
    #
    #     return self
    #
    # def transform(self,X,y=None):
    #     for i in X.columns:
    #         for j in range(len(X)):
    #             if abs(X[i][j]-self.mean[i])>(3*self.std[i]):
    #                 X[i][j]=999
    #
    #     return X
    #
    # def fit_transform(self,X,y=None):
    #     self.fit(X,y)
    #     return self.transform(X,y)

detector=outlierHandlingWithDev()
detector.fit(X_train)
detector.transform(X_train)
detector.transform(X_test)

X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

X_train=pd.DataFrame(X_train,columns=names)
X_test=pd.DataFrame(X_test,columns=names)

### Binning

### resample

In [108]:
from imblearn.over_sampling import SMOTE

smote=SMOTE()

X_train,y_train=smote.fit_resample(X_train,y_train)


### MinMaxScaler

In [109]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train[:]=scaler.fit_transform(X_train)
X_test[:]=scaler.transform(X_test)
X_train

,RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.370690,0.542087,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.0,0.000000,...,0.455968,0.460253,0.638994,0.587399,0.003439,0.067796,0.002780,0.212187,0.046105,0.009190
1,0.741379,0.247337,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,...,0.432894,0.436964,0.546212,0.127545,0.020124,0.033764,0.005696,0.088502,0.008232,0.020419
2,0.767241,0.777344,1.000000,0.000000,1.000000,0.000000,0.500000,0.000000,0.0,0.000000,...,0.389308,0.392968,0.563218,0.022627,0.721885,0.036499,0.005139,0.015420,0.000211,0.018433
3,0.129310,0.459340,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.0,1.000000,...,0.546080,0.551214,0.513451,0.241711,0.010123,0.044997,0.004215,0.105641,0.011536,0.014063
4,0.387931,0.516790,1.000000,1.000000,0.000000,0.000000,0.500000,0.000000,0.0,1.000000,...,0.337877,0.341053,0.757515,0.551796,0.001268,0.107620,0.001713,0.315393,0.107677,0.006009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,0.409828,0.591029,0.000000,0.000000,0.400185,0.599815,1.000000,0.700093,0.0,0.400185,...,0.433183,0.437255,0.485163,0.422396,0.000912,0.087474,0.002994,0.780283,0.755564,0.010471
474,0.291917,0.490428,0.030038,0.030038,0.030038,0.969962,0.515019,0.030038,0.0,1.000000,...,0.592461,0.598030,0.224649,0.150738,0.002628,0.003706,0.182378,0.028214,0.007319,0.588373
475,0.378168,0.773216,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,0.0,0.905624,...,0.101955,0.102913,0.458954,0.294714,0.000795,0.049888,0.005152,0.690485,0.597317,0.016790
476,0.381968,0.466615,0.198520,0.198520,0.198520,0.801480,0.599260,0.198520,0.0,1.000000,...,0.494050,0.498695,0.315029,0.169256,0.002865,0.018615,0.151092,0.100124,0.047363,0.487550


### PCA

In [110]:
def is_related(pair1, pair2):
    if (pair1[0] in pair2 or pair1[1] in pair2) and pair1!=pair2 and not (pair1[0]==pair2[1] and pair1[1]==pair2[0]):
        return True
    return False

def add_relation(corr_fea):
    for i in range(len(corr_fea)):
        for j in range(i,len(corr_fea)):
            if i!=j:
                if is_related(corr_fea[i],corr_fea[j]):
                    fea1=tuple(set(corr_fea[i])-set(corr_fea[j]))
                    fea2=tuple(set(corr_fea[j])-set(corr_fea[i]))
                    fea=fea1+fea2
                    if fea not in corr_fea and rev(fea) not in corr_fea:
                        corr_fea.append((fea1+fea2))

def rev(fea):
    f1=fea[0]
    f2=fea[1]

    return (f2,f1)

corr=X_train.corr()
corr=corr.where(abs(corr)>0.9)
count=0
corr_fea=[]
for i in corr.columns:
    for j in corr.columns:
        if i!=j and abs(corr[i][j])>0.9 and ((i,j) not in corr_fea) and ((j,i) not in corr_fea):
            count+=1
            corr_fea.append((i,j))
            add_relation(corr_fea)


In [111]:
from sklearn.decomposition import PCA

new_num_fea=len(X_train.columns)-count

pca=PCA(n_components=new_num_fea)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)


### Feature Selection

In [112]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

selector=RFECV(SVC(kernel="linear"),step=2000,min_features_to_select=10)
X_train=selector.fit_transform(X_train,y_train)
X_test=selector.transform(X_test)
X_train

# from sklearn.feature_selection import S

array([[-4.35381412e-01,  1.09358880e-01,  5.88588002e-01, ...,
        -7.63587318e-03, -7.10771208e-03, -2.95793265e-02],
       [-1.03724378e+00,  3.23960533e-01,  1.26919693e+00, ...,
         2.50070012e-02, -2.55934937e-02, -5.69553311e-02],
       [-2.47742268e+00,  1.51527950e+00, -4.22775172e-01, ...,
        -9.11002724e-02, -4.96205702e-02,  2.78973442e-02],
       ...,
       [ 2.29170429e+00, -3.77099481e-01,  7.19067268e-01, ...,
         1.34276178e-02, -3.50646136e-02, -1.99981715e-02],
       [-9.08768481e-01, -1.21430527e+00,  1.22266588e+00, ...,
         2.96633966e-02, -1.38802275e-02,  4.16677115e-02],
       [ 1.00276930e+00, -1.73693065e+00, -7.03053680e-01, ...,
        -4.22872383e-02, -5.45876039e-03,  1.94340175e-03]])

In [113]:

# c=dataset.corr().abs()
# s=c.unstack()
# so=s.sort_values(kind="quicksort")





# delcol=[]
# c=dataset.corr().shape[0]

# names=dataset.corr().columns
#
# for i in range(c):
#     for j in range(i+1,c):
#         if dataset.corr().iloc[i,j]>0.8 :
#             delcol.append((names[i],names[j]))
# print(len(delcol))
#
# import numpy as np
# np.a
#
# for i in range(c):
#     count = 0
#     for j in range(c):
#         if dataset.corr().iloc[i,j]>0.8:
#             count += 1





# Methods

In [114]:
from sklearn.metrics import confusion_matrix
from sklearn.tree import  DecisionTreeClassifier
from sklearn.metrics import classification_report
clf=DecisionTreeClassifier(max_features=4,max_depth=5,class_weight='balanced')
clf.fit(X_train,y_train)
# clf.predict(X_test)

print(classification_report(y_test,clf.predict(X_test)))
# print(classification_report(y_train,clf.predict(X_train)))
confusion_matrix(y_test,clf.predict(X_test))

              precision    recall  f1-score   support

           0       0.76      0.42      0.54        60
           1       0.24      0.58      0.34        19

    accuracy                           0.46        79
   macro avg       0.50      0.50      0.44        79
weighted avg       0.63      0.46      0.49        79



array([[25, 35],
       [ 8, 11]], dtype=int64)

In [115]:



from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
clf=MLPClassifier(hidden_layer_sizes=(12,40),activation="relu",max_iter=200,batch_size=20)
# cross_val_score(clf,X_train,y_train,cv=3)


clf.fit(X_train,y_train)
print(classification_report(y_test,clf.predict(X_test)))
confusion_matrix(y_test,clf.predict(X_test))
confusion_matrix(y_train,clf.predict(X_train))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87        60
           1       0.60      0.47      0.53        19

    accuracy                           0.80        79
   macro avg       0.72      0.69      0.70        79
weighted avg       0.79      0.80      0.79        79



array([[239,   0],
       [  0, 239]], dtype=int64)

In [116]:
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(class_weight="balanced")
forest.fit(X_train,y_train)
confusion_matrix(forest.predict(X_test),y_test)

array([[55, 17],
       [ 5,  2]], dtype=int64)

In [117]:
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.neural_network import MLPClassifier
# clf=MLPClassifier(hidden_layer_sizes=(12,40,23,30),activation="relu",max_iter=500,batch_size=20)
# # cross_val_score(clf,X_train,y_train,cv=3)
# from sklearn.feature_selection import SequentialFeatureSelector
# sfs=SequentialFeatureSelector(clf,n_features_to_select="auto")
#
# sfs.fit(X_train,y_train)
# print(classification_report(y_test,sfs.predict(X_test)))
# confusion_matrix(y_test,sfs.predict(X_test))
# confusion_matrix(y_train,sfs.predict(X_train))